In [1]:
%cd ../

/Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu


In [2]:
# If you get a SciPy error when installing Emukit, build it from source:

# git clone https://github.com/amzn/Emukit.git
# cd Emukit
# pip install -r requirements/requirements.txt
# python setup.py develop

In [3]:
import numpy as np
import pickle
import emukit as ek
import GPy

from emukit.model_wrappers import GPyModelWrapper
from emukit.experimental_design.experimental_design_loop import ExperimentalDesignLoop
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.core.initial_designs import RandomDesign, latin_design
from emukit.experimental_design.acquisitions import ModelVariance, IntegratedVarianceReduction
from emukit.core.loop import UserFunctionWrapper
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop
from emukit.core.optimization import GradientAcquisitionOptimizer

from sumo_grid_simulation.grid_simulation import Simulator

### Pseudocode

Fitting a gaussian process to a simulator using Emukit takes the following form (taken from L48 lectures):

```
initialize GP with some randomly chosen points
while stopping condition is not met:
    compute candidate point(s) using GP and acquisition funciton (model_variance) -> new point
    evaluate this new point with our simulator/user function -> observation
    update model with new observation -> new GP
```

### Simulator

In [4]:
simulator = Simulator(end_time=300)

### User Function
 
This is the function we want to understand. Namedly, how are CO2 emissions and timeLoss (the time lost due to driving below the ideal speed) affected by the following parameters: 

```
gridSize: the size of the grid network where the simulation is carried out
edgeMaxSpeed: legal speed limit in m/s - this can be exceeded (11.11 == 40km/h)
maxSpeed: the absolute maximum velocity of any vehicle in m/s (55.55 == 200 km/h)
edgeLength: length of the roads between intersections in meters
numLanes: number of lanes per road
accel: The acceleration ability of vehicles in m/s^2.
```

*A complete list of parmeters analysed is discussed in our report.*

In [5]:
def user_function_time_loss(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray, N is the number of points to evaluate, M is the number of parameters per each point """
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function time loss called with {X.shape[0]} inputs to simulate')

    
    for gridSize, edgeMaxSpeed, maxSpeed, edgeLength, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        
        alpha = 0.005
        max_number_of_vehicles = ((gridSize - 1) * gridSize * 2 + 4 * gridSize) * edgeLength / 5
        period = 300/(max_number_of_vehicles * alpha)
        
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            maxSpeed      = maxSpeed,
            edgeLength    = edgeLength,
            numberOfLanes = int(numLanes),
            accel         = accel,
            trips_generator_period = period
        )
        # average time loss / average route duration
        result.append(s['timeLoss']/s['duration'])
        i += 1
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

### Model (GP)

Our surrogate model is our emulator. In this case, a gaussian process. I think that emukit handles categorical/discrete inputs using one-hot encodings.

#### Model Inputs

In [6]:
# gridSize = ContinuousParameter('gridSize', min_value=3, max_value=20)
# edgeMaxSpeed = ContinuousParameter('edgeMaxSpeed', min_value=1, max_value=25)
# maxSpeed = ContinuousParameter('maxSpeed', min_value=1, max_value=25)
# edgeLength = ContinuousParameter('edgeLength', min_value=30, max_value=200)
# numberOfLanes = DiscreteParameter('numberOfLanes', domain=[1,2,3])
# accel = ContinuousParameter('accel', 1., 6.)

# parameter_space = ParameterSpace([gridSize, edgeMaxSpeed, maxSpeed, edgeLength, numberOfLanes, accel])
import experimental_design.config as config
parameter_space = config.get_parameter_space()

#### Initialize Model / Emulator (GP)

In [7]:
#init_X, init_Y = pickle.load(open('experimental_design/init_points/25_init_points.pkl', 'rb'))

design = RandomDesign(parameter_space)  # initialize with random points
num_init_points = 20
init_X = design.get_samples(num_init_points)
init_Y = user_function_time_loss(init_X)
with open(f'experimental_design/init_points/timeloss_{num_init_points}_init_points.pkl', "wb") as f:
      pickle.dump((init_X, init_Y), f)


[[ 3.         24.66793403 18.65343132 45.80344501  1.          2.39439633]
 [10.         11.80333944 15.97638779 32.02350983  1.          2.56339982]
 [ 7.         10.19041561  8.76923382 63.33584664  3.          2.15555596]
 [14.         21.56751281 23.78228449 52.5676873   1.          2.41461906]
 [14.         15.54727024 11.94020297 42.92085795  1.          4.36404886]
 [ 8.         19.47900443 16.51981043 53.1400139   3.          2.53929929]
 [ 4.         20.7139968  20.7109454  44.81015629  1.          2.5658248 ]
 [13.         24.06038066 21.77722659 59.37430992  1.          2.01264096]
 [13.          9.83216652 11.29350474 34.73224017  2.          4.19603383]
 [15.         22.36833105 12.09319069 55.55293239  3.          3.97544349]
 [ 5.         10.96149185 18.96446876 66.30919818  2.          2.91070748]
 [ 7.         10.28085861 15.24043844 36.39597517  3.          4.64802667]
 [ 5.         16.74999664  6.16219104 31.28092607  2.          3.65698353]
 [11.          9.26046725

In [8]:
emulator = GPy.models.GPRegression(init_X, init_Y, noise_var=1e-10)
emukit_model = GPyModelWrapper(emulator, n_restarts=5)
emukit_model.optimize()
emulator

Optimization restart 1/5, f = -20.85228033618856
Optimization restart 2/5, f = -23.612755113465976
Optimization restart 3/5, f = 6.170076616627577
Optimization restart 4/5, f = 6.170076616627634
Optimization restart 5/5, f = 6.170076616626833


GP_regression.,value,constraints,priors
rbf.variance,0.11918608925535737,+ve,
rbf.lengthscale,40.47522433726006,+ve,
Gaussian_noise.variance,0.001442893577660121,+ve,


### Optimization

#### Acquisition Function

In [9]:
model_variance = ModelVariance(model=emukit_model)

#### Optimizer

In [10]:
optimizer = GradientAcquisitionOptimizer(parameter_space)

#### Experimental Design

In [11]:
ed_loop = ExperimentalDesignLoop(
    model = emukit_model,
    space = parameter_space,
    acquisition = model_variance,
    acquisition_optimizer = optimizer,
)

In [12]:
ed_loop.run_loop(user_function_time_loss, 200)

Optimization restart 1/5, f = -23.61275511362676
Optimization restart 2/5, f = 6.170076616634351
Optimization restart 3/5, f = -23.612755114103486
Optimization restart 4/5, f = -23.61275511365549
Optimization restart 5/5, f = -23.612755114075018
[[20.  25.  27.  30.   3.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -24.249395224098528
Optimization restart 2/5, f = 7.6711342630196935
Optimization restart 3/5, f = 7.671134984736305
Optimization restart 4/5, f = -24.249395224583893
Optimization restart 5/5, f = 7.671134984702128
[[20.         8.         5.        70.         1.         3.4960138]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -23.865597972099124
Optimization restart 2/5, f = -23.86559797209872
Optimization restart 3/5, f = -23.865597972072095
Optimization restart 4/5, f = -23.865597

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


 Retrying in 1 seconds
Optimization restart 1/5, f = -31.24022732338466
Optimization restart 2/5, f = -31.240227323405627
Optimization restart 3/5, f = -31.2402273233512
Optimization restart 4/5, f = 7.790928140663183
Optimization restart 5/5, f = 7.790928139641878
[[ 3.         25.          5.         51.22417494  1.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -31.65375717094895
Optimization restart 2/5, f = -31.65375717102014
Optimization restart 3/5, f = -31.65375717101681
Optimization restart 4/5, f = 7.884477210273204
Optimization restart 5/5, f = 7.884477215137368
[[ 3. 25. 27. 30.  1.  5.]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -32.91232720078702
Optimization restart 2/5, f = 9.523239460897734
Optimization restart 3/5, f = -32.912327200811625
Optimization restart 4/5

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


 Retrying in 1 seconds
Optimization restart 1/5, f = -36.691509076569716
Optimization restart 2/5, f = 10.911471152582571
Optimization restart 3/5, f = 10.911471152582582
Optimization restart 4/5, f = 10.911471152582568
Optimization restart 5/5, f = 10.91147070025006
[[20.  8.  5. 30.  1.  5.]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -37.11009951065618
Optimization restart 2/5, f = 11.0061149574931
Optimization restart 3/5, f = 11.006114059308125
Optimization restart 4/5, f = 11.006114957382614
Optimization restart 5/5, f = -37.110099516278126
[[ 3.  8.  5. 70.  3.  5.]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -37.155718480023914
Optimization restart 2/5, f = -37.1557184801193
Optimization restart 3/5, f = -37.155718480071386
Optimization restart 4/5, f = -37.15571848011919
Optimization restar

...

 Retrying in 1 seconds
Optimization restart 1/5, f = -214.0512367289282
Optimization restart 2/5, f = -214.0512367260657
Optimization restart 3/5, f = 36.59520052894288
Optimization restart 4/5, f = -214.05123672891602
Optimization restart 5/5, f = 36.59520048870017
[[20.        13.5654532  5.        70.         3.         5.       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -216.21422125840678
Optimization restart 2/5, f = -172.80130249002985
Optimization restart 3/5, f = -216.21432308058917
Optimization restart 4/5, f = -172.80119681169123
Optimization restart 5/5, f = 36.68328569773497
[[ 3.         16.6602456  27.         55.06948475  1.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -217.74180986824223
Optimization restart 2/5, f = -217.74180986812814
Optimization restart 3/

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 5/5, f = -228.6170297059583
[[ 3.          8.          5.         66.04257547  3.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -230.8265434622685
Optimization restart 2/5, f = -187.26222061386133
Optimization restart 3/5, f = 40.357834041973405
Optimization restart 4/5, f = 40.357834166585945
Optimization restart 5/5, f = -230.82674140272348
[[20.          8.         16.66450968 30.          1.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -232.74815799105957
Optimization restart 2/5, f = -232.74815799133646
Optimization restart 3/5, f = -232.7481579903687
Optimization restart 4/5, f = -189.51499753892986
Optimization restart 5/5, f = -189.5150141685103
[[20.         14.00391133 20.03430788 42.71756918  3.          5.        ]]

User functi

[[ 3.         25.         27.         47.69609761  1.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -271.29057432980954
Optimization restart 2/5, f = -223.1780240518786
Optimization restart 3/5, f = 43.964977244339025
Optimization restart 4/5, f = 43.964977860710945
Optimization restart 5/5, f = -223.17802693874046
[[10.         16.13647857 27.         30.          1.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -273.19944924306617
Optimization restart 2/5, f = -223.84983746933096
Optimization restart 3/5, f = -273.19944925938694
Optimization restart 4/5, f = -223.849838068973
Optimization restart 5/5, f = -273.19944922132777
[[20.         25.         15.60012719 30.          3.          5.        ]]

User function time loss called with 1 inputs to simulate

Ev

 Retrying in 1 seconds
Optimization restart 1/5, f = -304.77644029023554
Optimization restart 2/5, f = -304.77644028981615
Optimization restart 3/5, f = 48.693223708875564
Optimization restart 4/5, f = -263.1394143065676
Optimization restart 5/5, f = -263.1394129302505
[[13.          8.         21.89791688 30.          3.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -306.0910915374649
Optimization restart 2/5, f = -306.0910915328957
Optimization restart 3/5, f = -264.67474786505363
Optimization restart 4/5, f = -306.0910915377405
Optimization restart 5/5, f = -306.0910915372598
[[20.         17.91278924 22.20420628 52.27423754  3.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -308.5662193947491
Optimization restart 2/5, f = 49.212185824657126
Optimization resta

In [13]:
#emukit_model = pickle.load(open('experimental_design/models/5_init_50_loop_model_timeLoss.pkl', 'rb'))

### Evaluate

#### Test on 50 point test set

In [13]:
#test_X, test_Y = pickle.load(open('experimental_design/test_points/25_test_points.pkl', 'rb'))


design = RandomDesign(parameter_space)  # initialize with random points
num_test_points = 50
test_X = design.get_samples(num_test_points)
test_Y = user_function_time_loss(test_X)
with open(f'experimental_design/test_points/timeloss_{num_test_points}_test_points.pkl', "wb") as f:
      pickle.dump((test_X, test_Y), f)


[[17.         15.9609046  20.31909159 58.51464048  1.          3.89247569]
 [13.         18.5089897  15.84031679 60.58431295  1.          4.51871503]
 [ 6.         15.63778625 23.23192495 38.21855506  2.          4.36462192]
 [ 8.         21.47122457 20.2723775  31.59288103  1.          1.73708032]
 [ 7.          8.1305096  14.71677518 47.1115762   1.          4.05969823]
 [17.         24.41556527 11.09721266 48.85821293  1.          2.14338255]
 [20.         18.50034083  9.22673622 51.59033242  3.          1.84518383]
 [ 7.         17.86125616 13.46564453 69.97254571  1.          2.0695198 ]
 [14.         21.20653145 18.66722811 34.87322314  2.          2.8267309 ]
 [11.         15.55019235 13.80279944 41.36560422  3.          3.36362474]
 [19.         24.08040108 13.74524842 68.14360099  2.          4.45271898]
 [11.          8.3386272  22.44392573 35.26618074  3.          4.13267972]
 [ 5.         21.62964061 17.17844143 36.80139729  3.          4.92450197]
 [18.         10.82466882

In [14]:
pred_Y, pred_Y_variance = emukit_model.predict(test_X)
mse = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Root mean squared error: ', mse)

Root mean squared error:  0.038576700913822


In [15]:
pred_Y*100, test_Y*100

(array([[29.39431479],
        [27.5630598 ],
        [42.48251326],
        [57.79179071],
        [18.88173615],
        [21.8593178 ],
        [18.04391826],
        [28.51250488],
        [46.77777156],
        [30.13972291],
        [26.15417981],
        [23.15526092],
        [42.90486767],
        [19.31260738],
        [45.78644501],
        [15.55296633],
        [41.3478749 ],
        [14.49640577],
        [17.33232183],
        [16.47809204],
        [40.84029004],
        [18.55490361],
        [24.78606731],
        [47.66526106],
        [45.98045894],
        [34.94062624],
        [21.86004841],
        [23.65928162],
        [29.88431662],
        [42.59417804],
        [20.22102418],
        [18.30134391],
        [23.86646849],
        [22.85966326],
        [47.99124958],
        [19.83182509],
        [38.18459081],
        [31.75678855],
        [29.79181813],
        [44.89784004],
        [18.5313934 ],
        [33.91458334],
        [22.71447341],
        [31

### Save Model

In [17]:
num_init_points = 20
num_loop_iters = 200


with open(f'experimental_design/models/{num_init_points}_init_{num_loop_iters}_loop_model_timeLoss.pkl', "wb") as f:
     pickle.dump(emukit_model, f)

## Baseline against a GP fitted to 200 random points

In [17]:
design = RandomDesign(parameter_space)  # initialize with random points
num_init_points = 200
init_X = design.get_samples(num_init_points)
init_Y = user_function_time_loss(init_X)
with open(f'experimental_design/init_points/timeloss_{num_init_points}_init_points.pkl', "wb") as f:
      pickle.dump((init_X, init_Y), f)

[[11.         18.31701021 25.8069197  65.86348662  1.          4.98939197]
 [20.         13.56188998 19.99389044 42.84173812  1.          4.32947393]
 [13.         17.72679139 20.33547546 37.08075159  1.          2.30604458]
 ...
 [ 9.          9.23165358 10.20133108 34.92039336  2.          3.44826594]
 [17.         10.32134816 13.80464484 39.55941328  3.          2.65482971]
 [ 9.          9.45715526 25.26987995 64.10234357  1.          2.98995016]]

User function time loss called with 200 inputs to simulate

Evaluating input: 1 of 200

 Retrying in 1 seconds

Evaluating input: 2 of 200

 Retrying in 1 seconds

Evaluating input: 3 of 200

 Retrying in 1 seconds

Evaluating input: 4 of 200

 Retrying in 1 seconds

Evaluating input: 5 of 200

 Retrying in 1 seconds

Evaluating input: 6 of 200

 Retrying in 1 seconds

Evaluating input: 7 of 200

 Retrying in 1 seconds

Evaluating input: 8 of 200

 Retrying in 1 seconds

Evaluating input: 9 of 200

 Retrying in 1 seconds

Evaluating inpu

 Retrying in 1 seconds

Evaluating input: 146 of 200

 Retrying in 1 seconds

Evaluating input: 147 of 200

 Retrying in 1 seconds

Evaluating input: 148 of 200

 Retrying in 1 seconds

Evaluating input: 149 of 200

 Retrying in 1 seconds

Evaluating input: 150 of 200

 Retrying in 1 seconds

Evaluating input: 151 of 200

 Retrying in 1 seconds

Evaluating input: 152 of 200

 Retrying in 1 seconds

Evaluating input: 153 of 200

 Retrying in 1 seconds

Evaluating input: 154 of 200

 Retrying in 1 seconds

Evaluating input: 155 of 200

 Retrying in 1 seconds

Evaluating input: 156 of 200

 Retrying in 1 seconds

Evaluating input: 157 of 200

 Retrying in 1 seconds

Evaluating input: 158 of 200

 Retrying in 1 seconds

Evaluating input: 159 of 200

 Retrying in 1 seconds

Evaluating input: 160 of 200

 Retrying in 1 seconds

Evaluating input: 161 of 200

 Retrying in 1 seconds

Evaluating input: 162 of 200

 Retrying in 1 seconds

Evaluating input: 163 of 200

 Retrying in 1 seconds

Eval

In [18]:
emulator_rnd = GPy.models.GPRegression(init_X, init_Y, noise_var=1e-10)
emukit_model_rnd = GPyModelWrapper(emulator_rnd, n_restarts=200)
emukit_model_rnd.optimize()
emulator_rnd

Optimization restart 1/200, f = -276.5026637281492
Optimization restart 2/200, f = -365.7520421318908
Optimization restart 3/200, f = -365.7520421314708
Optimization restart 4/200, f = 40.66828345782608
Optimization restart 5/200, f = -365.7520421317804
Optimization restart 6/200, f = -365.75204211436096
Optimization restart 7/200, f = -365.75204212797524
Optimization restart 8/200, f = 40.66827659905066
Optimization restart 9/200, f = -365.75204213168627
Optimization restart 10/200, f = -365.7520420868319
Optimization restart 11/200, f = 40.668284087169496
Optimization restart 12/200, f = -365.752042127143
Optimization restart 13/200, f = 40.66828441955927
Optimization restart 14/200, f = -365.75204210551396
Optimization restart 15/200, f = 40.668284419631874
Optimization restart 16/200, f = -365.7520421325222
Optimization restart 17/200, f = -365.7520421325106
Optimization restart 18/200, f = -365.75204213106724
Optimization restart 19/200, f = 40.66828441855763
Optimization restart 

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 62/200, f = -358.4654200803495
Optimization restart 63/200, f = 40.66828441962947
Optimization restart 64/200, f = 40.668284419630936
Optimization restart 65/200, f = -365.75204213252346
Optimization restart 66/200, f = -314.94744005043424
Optimization restart 67/200, f = -365.75204213252766
Optimization restart 68/200, f = -365.7520421324208
Optimization restart 69/200, f = 40.66828441963099
Optimization restart 70/200, f = -365.7520421311105
Optimization restart 71/200, f = -365.7520421248507
Optimization restart 72/200, f = -365.75204212933045
Optimization restart 73/200, f = -365.75204213246786
Optimization restart 74/200, f = 40.668284418513224
Optimization restart 75/200, f = -365.75204211120746
Optimization restart 76/200, f = -365.7520411304498
Optimization restart 77/200, f = -365.7520421263308
Optimization restart 78/200, f = -365.7520421317861
Optimization restart 79/200, f = -365.7520421323662
Optimization restart 80/200, f = -365.752042132099
Optimizat

GP_regression.,value,constraints,priors
rbf.variance,0.07057955168730747,+ve,
rbf.lengthscale,19.776866680495676,+ve,
Gaussian_noise.variance,0.0006994875579710608,+ve,


In [19]:
pred_Y, pred_Y_variance = emukit_model_rnd.predict(test_X)
mse = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Root mean squared error: ', mse)

Root mean squared error:  0.041844548762875924


In [20]:
1- (0.038576700913822 / 0.041844548762875924)

# Experimental design is ~8% better than random.

0.07809494774509607